## Building an image classification model using very little data
Based on the tutorial by Francois Chollet @fchollet https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html and the workbook by Guillaume Dominici https://github.com/gggdominici/keras-workshop

This tutorial presents several ways to build an image classifier using keras from just a few hundred or thousand pictures from each class you want to be able to recognize.

We will go over the following options:
 * training a small network from scratch (as a baseline)
 * using the bottleneck features of a pre-trained network
 * fine-tuning the top layers of a pre-trained network

This will lead us to cover the following Keras features:

* fit_generator for training Keras a model using Python data generators
* ImageDataGenerator for real-time data augmentation
* layer freezing and model fine-tuning
* ...and more.

# Data loading

In [1]:
##This notebook is built around using tensorflow as the backend for keras
!pip install pillow
!KERAS_BACKEND=tensorflow python -c "from keras import backend"

Using TensorFlow backend.


In [2]:
##Updated to Keras 2.0
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras import applications
from keras.models import Model

os.chdir('/Users/z00193k/Desktop/ML_AI_DL/CNN/')

Using TensorFlow backend.


In [3]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

# Imports

In [4]:
##preprocessing
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

Found 2048 images belonging to 2 classes.
Found 824 images belonging to 2 classes.


# Small Conv Net

## Model architecture definition

In [6]:
# a simple stack of 3 convolution layers with a ReLU activation and followed by max-pooling layers.
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [7]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [14]:
epochs = 30
train_samples = 2048
validation_samples = 832

In [15]:
model.fit_generator(
        train_generator,
        steps_per_epoch=train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_samples// batch_size,)
#About 60 seconds an epoch when using CPU

Epoch 1/30
64/64 [==============================] - 50s - loss: 0.6199 - acc: 0.6582 - val_loss: 0.5838 - val_acc: 0.7027
Epoch 2/30
64/64 [==============================] - 57s - loss: 0.5864 - acc: 0.6948 - val_loss: 0.5901 - val_acc: 0.6663
Epoch 3/30
64/64 [==============================] - 61s - loss: 0.5156 - acc: 0.7407 - val_loss: 0.5731 - val_acc: 0.6881
Epoch 4/30
64/64 [==============================] - 52s - loss: 0.4990 - acc: 0.7588 - val_loss: 0.6906 - val_acc: 0.6626
Epoch 5/30
64/64 [==============================] - 54s - loss: 0.4532 - acc: 0.7817 - val_loss: 0.5914 - val_acc: 0.7087
Epoch 6/30
64/64 [==============================] - 53s - loss: 0.4169 - acc: 0.8042 - val_loss: 0.6900 - val_acc: 0.6942
Epoch 7/30
64/64 [==============================] - 53s - loss: 0.3887 - acc: 0.8267 - val_loss: 0.5766 - val_acc: 0.7318
Epoch 8/30
64/64 [==============================] - 53s - loss: 0.3388 - acc: 0.8540 - val_loss: 0.6191 - val_acc: 0.7439
Epoch 9/30
64/64 [======

In [16]:
model.save_weights('models/basic_cnn_1_epochs.h5')

In [17]:
model.evaluate_generator(validation_generator, validation_samples)

[1.8083927611124169, 0.71962226939939178]